<a href="https://colab.research.google.com/github/Ajay-2007/deep-learning-projects/blob/master/12.saving_and_loading_model_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
scipy.__version__

'1.1.0'

In [0]:
from scipy.misc import imread, imresize

In [1]:
!pip install scipy==1.1.0

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from scipy.misc import imread, imresize
# import imageio
# from PIL import Image

In [0]:
def process_image(image):
  img = imread(image)
  img = imresize(img, (256, 256)) # 256 x 256 x 3
  img = img.transpose(2, 0, 1)  # channels first
  img = img / 255.  
  img = torch.FloatTensor(img).cuda()
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  transform = transforms.Compose([normalize])
  img = transform(img) # (3, 256, 256)
  return img

In [0]:
# Define Model
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # in_channels = 3, out_channels = 6, filter_size = 5, 0 padding, stride 1
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 12, 5)
    self.fc1 = nn.Linear(12*61*61, 120)
    self.fc2 = nn.Linear(120, 10)

  def forward(self, x):
    # consider an image of size 256x256
    x = F.relu(self.conv1(x))       # output_size = [ (256 - 5 + 2 (0)) / 1] + 1 --> 252x252
    x = self.pool(x)                # output_size = 252/2 --> 126x125
    x = F.relu(self.conv2(x))       # output_size = [(126 - 5 + 2 (0)) / 1] + 1 --> 122x122
    x = self.pool(x)                # output_size = 122/2 --> 61x61
    x = x.view(-1, 12*61*61)        # (1, 44652)
    x = F.relu(self.fc1(x))         # (1, 120)
    x = self.fc2(x)                 # 
    return x


In [0]:
# Initialize model
model = CNN()
model = model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [34]:
image = process_image('test.png')
image = image.unsqueeze(0)    # batch dimension

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
output = model.forward(image)
print(output.shape)

torch.Size([1, 10])


In [36]:
# Print model's state_dict
print('Model\'s state dict: ')
for param_tensor in model.state_dict():
  print(param_tensor, '\t', model.state_dict()[param_tensor].size())

Model's state dict: 
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([12, 6, 5, 5])
conv2.bias 	 torch.Size([12])
fc1.weight 	 torch.Size([120, 44652])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([10, 120])
fc2.bias 	 torch.Size([10])


In [0]:
torch.save(model.state_dict(), 'model.pth.tar')

In [38]:
model = CNN()
model.load_state_dict(torch.load('model.pth.tar'))
model.eval()    # Set dropout and batch normalization layers to evaluation mode before inference (testing)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

### Saving and Loading a General Checkpoint for inference and/or Resuming Training

In [0]:
checkpoint = {'epoch':1,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': 0.2}
torch.save(checkpoint, 'model.pth.tar')

In [0]:
model = CNN().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
 

In [0]:
checkpoint = torch.load('model.pth.tar')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [42]:
# If testing
model.eval()

# If training, resume training
model.train()

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)